### Crytocurrencies analyis

#### Objective
Show statistical analysis on
- Random variable 'winning', where X='BTC' is the event where BTC price increase the most in pct compared to other ccys during in interval
- Random varible 'no trade', where X='BTC' is the event where BTC was not traded during an interval

#### Approach
Just explore data by counting for now 


In [83]:
# Non tech settings
analysis_interval = 20 # seconds
analysis_period_start = '2017-07-01T00:00:00Z'
analysis_period_end = '2017-07-03T00:02:00Z'
ccys = ['BTCUSD', 'ETHUSD']

#### Dependencies

In [84]:
# Tech
import os
import sys
import pandas as pd
import numpy as np
import pendulum

module_path = os.path.join('..')
if module_path not in sys.path:
    sys.path.append(module_path)

from src.models import DeltaPoint, Trade    
from src.db import PgDb

db = PgDb()

###### Retrieve raw price points

In [87]:
# From db
data = [(ccy, db.get_pricepoints_20(ccy, analysis_period_start, analysis_period_end)) for ccy in ccys]

# To dataframe
df_dict = {}
for ccy, values in data:
    if not df_dict.get('time'):
        df_dict['time'] = [p['time'] for p in values]
        
    prefix = ccy.replace('USD', '')
    df_dict[f'{prefix}'] = [float(p['pct']) for p in values]
    df_dict[f'{prefix}_trades_count'] = [float(p['n']) for p in values]
    
df = pd.DataFrame(df_dict)
df['winner'] = df[['BTC', 'ETH']].idxmax(axis=1)
df.head(20)

,time,BTC,BTC_trades_count,ETH,ETH_trades_count,winner
0,2017-07-01 19:59:40+03:00,-0.20415,3.0,-0.01556,6.0,ETH
1,2017-07-01 20:00:00+03:00,0.05010,1.0,0.07277,2.0,ETH
2,2017-07-01 20:00:20+03:00,-0.02504,5.0,0.35211,7.0,ETH
3,2017-07-01 20:00:40+03:00,-0.01252,4.0,-0.30892,4.0,BTC
4,2017-07-01 20:01:00+03:00,0.02922,3.0,-0.09947,14.0,BTC
5,2017-07-01 20:01:20+03:00,-0.00417,1.0,0.10340,7.0,ETH
6,2017-07-01 20:01:40+03:00,-0.02504,3.0,-0.00383,3.0,ETH
7,2017-07-01 20:02:00+03:00,-0.05844,2.0,0.30223,2.0,ETH
8,2017-07-01 20:02:20+03:00,0.04177,3.0,-0.06103,19.0,BTC
9,2017-07-01 20:02:40+03:00,0.00418,2.0,-0.16411,8.0,BTC


###### Winning counts

In [88]:
winners = df['winner']
counts = winners.value_counts()
probs = counts / winners.size

stats = pd.DataFrame([counts, probs], index=['Count', 'Probability'])
stats.head()

,BTC,ETH
Count,3067.000000,2521.000000
Probability,0.548855,0.451145


###### No trade counts

In [107]:
no_trade_list = []
for ccy in ccys:
    prefix = ccy.replace('USD', '')
    column = prefix + '_trades_count'
    trades_count = df[column]
    aggregated_trade_counts = trades_count.value_counts()
    no_trades_count = aggregated_trade_counts.get(0.0)
    if not no_trades_count:
        no_trades_count = 0
        
    ratio = no_trades_count / trades_count.size
        
    no_trade_list.append((prefix, no_trades_count, trades_count.size, ratio))
        
no_trade_df = pd.DataFrame(no_trade_list, columns=['Ccy', 'No Trade Count', 'Total', 'Ratio'])
no_trade_df.head()

,Ccy,No Trade Count,Total,Ratio
0,BTC,806,5588,0.144238
1,ETH,654,5588,0.117037
